In [137]:
import json
import pandas as pd
from collections import defaultdict
from glob import glob
from os import path
from datetime import datetime

In [122]:
FILES_DIR = '/home/skussl/Downloads/bond-files/' # directory with ogg files from archive (files from Bond)

In [123]:
files = [path.basename(x) for x in glob(path.join(FILES_DIR, 'files/*'))]

In [124]:
with open('../youtube-api/videos_list_yutt.json') as f:
    videos_list_yutt = json.loads(f.read())

In [133]:
found = set()
canditates = defaultdict(list)

for file in files:
    dt_file = datetime.strptime(file[:8], "%Y%m%d")
    name = file[9:-4]

    for video in videos_list_yutt:
        dt = datetime.strptime(video['snippet']['publishedAt'][:10], "%Y-%m-%d")
        
        dt_diff = (dt_file - dt).days
        
        if abs(dt_diff) > 1:
            continue
            
        canditates[file].append(video)
        
        if video['snippet']['title'].replace('?', '') in [name, name.replace(' -', ':', 1).replace('_', ' ')]:
            found.add(file)
            
            if file in canditates:
                del canditates[file]
            
            break

In [139]:
not_found = set(files) - found
not_found = sorted(not_found)[::-1]

In [140]:
len(not_found)

108

In [142]:
with open('files_not_found.json', 'w') as f:
    f.write(json.dumps(not_found))

In [143]:
canditate_list = []

for file in not_found:
    if not file in canditates:
        canditate_list.append({
            'filename': file
        })
    else:
        for video in canditates[file]:
            canditate_list.append({
                'filename': file,
                'candidateTitle': video['snippet']['title'],
                'candidateLink': 'https://youtu.be/' + video['id']
            })

In [145]:
df = pd.DataFrame(canditate_list)

In [147]:
df.to_excel('not_found_candidates.xlsx')